In [18]:
#เรียก library ที่จะใช้
import face_recognition
import cv2
import numpy as np

In [19]:
#ดึงวิดีโอตัวอย่างเข้ามา, ถ้าต้องการใช้webcamให้ใส่เป็น0
video_capture = cv2.VideoCapture("sample2.mp4")

# Load ใบหน้าคนที่ต้องการรู้จำ #คนที่1
lufi_image = face_recognition.load_image_file("lufi-m.jpg")
lufi_face_encoding = face_recognition.face_encodings(lufi_image)[0]

# Load ใบหน้าคนที่ต้องการรู้จำ #คนที่2
zoro_image = face_recognition.load_image_file("zoro-m.jpg")
zoro_face_encoding = face_recognition.face_encodings(zoro_image)[0]

# Load ใบหน้าคนที่ต้องการรู้จำ #คนที่3
sanji_image = face_recognition.load_image_file("sanji.jpg")
sanji_face_encoding = face_recognition.face_encodings(sanji_image)[0]

# Load ใบหน้าคนที่ต้องการรู้จำ #คนที่4
usob_image = face_recognition.load_image_file("usob.jpg")
usob_face_encoding = face_recognition.face_encodings(usob_image)[0]

# Load ใบหน้าคนที่ต้องการรู้จำ #คนที่5
nami_image = face_recognition.load_image_file("nami.jpg")
nami_face_encoding = face_recognition.face_encodings(nami_image)[0]


# สร้าง array เก็บชื่อ
known_face_encodings = [lufi_face_encoding,zoro_face_encoding,sanji_face_encoding,usob_face_encoding,nami_face_encoding]
known_face_names = ["Lufi","Zoro","Sanji","Usob","Nami"]

# ประกาศตัวแปร 
face_locations = []
face_encodings = []
face_names = []

#กำหนดตัวแปรสำหรับคิดเฟรมเว้นเฟรมเพื่อเพิ่ม fps 
process_this_frame = True
video_ended = False

#loopคำนวณแต่ละเฟรมของวิดีโอ
while True:
    # อ่านค่าแต่ละเฟรมจากวิดีโอ
    ret, frame = video_capture.read()
    if not ret:
        # ถ้าอ่านเฟรมไม่ได้แสดงว่าวิดีโอจบแล้ว
        video_ended = True
        break
    # Only process every other frame of video to save time
    if process_this_frame:
        # ลดขนาดภาพเป็น 1/4 เท่า เพื่อเพิ่ม fps และทำงานไวขึ้น 
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])   
        
        #ค้นหาตำแหน่งใบหน้าในเฟรม 
        face_locations = face_recognition.face_locations(rgb_small_frame)

        #นำใบหน้ามาหา features ต่าง ๆ ที่เป็นเอกลักษณ์ 
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        #เทียบแต่ละใบหน้า
        face_names = []
        for face_encoding in face_encodings:
            # หาหน้าที่ match กับหน้าต้นแบบ
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    #สลับค่าเป็นค่าตรงข้ามเพื่อให้คิดเฟรมเว้นเฟรม
    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # ขนาด ที่จะค้นหาใบหน้า (scaled to 1/4 size)
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    
    cv2.imshow('sample2', frame)
    if video_ended:
        pass
    

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [20]:
# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()